In [1]:
import torch
from gym_runner.gym_runner import GymRunner
from gym_runner.q_func_approx import QFuncSmallThreelayer, QFuncFree
from gym_runner.agents.sarsa_agent import SarsaAgent
from gym_runner.agents.q_learning_agent import QLearningAgent, QLearningAgentExperienceReplay
import pandas as pd
import altair as alt



In [2]:
runner = GymRunner("CartPole-v1", display_metrics=True)



In [3]:
num_actions = runner.env.action_space.n


In [4]:
num_states = runner.env.observation_space.shape[0]
num_states


4

In [5]:
q_func = QFuncSmallThreelayer(
    num_states=num_states,
    num_actions=num_actions,
    optimizer="SGD",
    loss_func="l1",
    alpha=0.001,
)

q_func = QFuncFree(
    num_states=num_states,
    num_actions=num_actions,
    model_layers = [
        torch.nn.Linear(num_states, 24),
        torch.nn.Tanh(),
        torch.nn.Linear(24, 48),
        torch.nn.Tanh(),
        torch.nn.Linear(48, num_actions)
    ],
    optimizer="SGD",
    loss_func="l1",
    alpha=0.001,
)


In [6]:
q_func_Q = QFuncSmallThreelayer(
    num_states=num_states,
    num_actions=num_actions,
    optimizer='Adam',
    loss_func='mse',
    alpha=0.01,
)


In [7]:
q_func_Q_exp = QFuncSmallThreelayer(
    num_states=num_states,
    num_actions=num_actions,
    optimizer='Adam',
    loss_func='mse',
    alpha = 0.01
)

In [8]:
agent = SarsaAgent(
    q_func_approx=q_func,
    num_states=num_states,
    num_actions=num_actions,
    gamma=0.9225,
    epsilon=1.0,
    epsilon_decay=0.9975,
)


In [9]:
agent_Q = QLearningAgent(
    q_func_approx=q_func_Q,
    num_states=num_states,
    num_actions=num_actions,
    gamma=0.9225,
    epsilon=1.0,
    epsilon_decay=0.9975,
)


In [6]:
agent_Q_exp = QLearningAgentExperienceReplay(
    q_func_approx=q_func,
    num_states=num_states,
    num_actions=num_actions,
    gamma=0.9225,
    epsilon=1.0,
    epsilon_decay=0.9975,
)


In [11]:
rewards = runner.train(agent=agent, num_episodes=1000, plot_state=False)



Epsilon:  0.1
Current Reward:  9.0
Episode:  990


In [12]:
rewards_Q = runner.train(agent=agent_Q, num_episodes=1000, plot_state=False)



Epsilon:  0.1
Current Reward:  9.0
Episode:  990


In [7]:
rewards_Q_exp = runner.train(agent=agent_Q_exp, num_episodes=1000, plot_state=False)



Epsilon:  0.1
Current Reward:  11.0
Episode:  990


In [14]:
test_rewards = runner.attempt(agent, num_episodes=100)


In [15]:
test_rewards_Q = runner.attempt(agent_Q, num_episodes=100)


In [16]:
test_rewards_Q_exp = runner.attempt(agent_Q_exp, num_episodes=100)


In [17]:
rewards = pd.DataFrame(rewards).reset_index()
rewards.columns = ["episode", "reward"]

test_rewards = pd.DataFrame(test_rewards).reset_index()
test_rewards.columns = ["episode", "reward"]

rewards_Q = pd.DataFrame(rewards_Q).reset_index()
rewards_Q.columns = ["episode", "reward"]

test_rewards_Q = pd.DataFrame(test_rewards_Q).reset_index()
test_rewards_Q.columns = ["episode", "reward"]

rewards_Q_exp = pd.DataFrame(rewards_Q_exp).reset_index()
rewards_Q_exp.columns = ["episode", "reward"]

test_rewards_Q_exp = pd.DataFrame(test_rewards_Q_exp).reset_index()
test_rewards_Q_exp.columns = ["episode", "reward"]


In [18]:
alt.Chart(rewards).mark_point().encode(x="episode", y="reward") | alt.Chart(
    test_rewards
).mark_point().encode(x="episode", y="reward")


alt.HConcatChart(...)

In [19]:
alt.Chart(rewards_Q).mark_point().encode(x="episode", y="reward") | alt.Chart(
    test_rewards_Q
).mark_point().encode(x="episode", y="reward")


alt.HConcatChart(...)

In [20]:
alt.Chart(rewards_Q_exp).mark_point().encode(x="episode", y="reward") | alt.Chart(
    test_rewards_Q_exp
).mark_point().encode(x="episode", y="reward")


alt.HConcatChart(...)